In [ ]:
import gradio as gr
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as T
import pandas as pd
import cv2
import os

# Завантаження моделі
from model import UNet
MODEL_PATH = "models/unet_model.pth"
model = UNet(in_channels=1, out_channels=1)
model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu')))
model.eval()

# Завантаження правил із MnM (CSV з правилами для кожного пацієнта)
RULES_DF = pd.read_csv("rules.csv") 

transform = T.Compose([
    T.Grayscale(),
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize(mean=[0.5], std=[0.5])
])

def get_rules_for_image(filename: str):
    basename = os.path.basename(filename)
    row = RULES_DF[RULES_DF['filename'] == basename]
    if not row.empty:
        threshold = int(row.iloc[0]['threshold'])
        morphology = row.iloc[0]['morphology']
        return threshold, morphology
    return 128, 'none'  

def segment_mri(image: Image.Image, use_rules: bool):
    input_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)
        mask = torch.sigmoid(output).squeeze().numpy()

    # Базове бінаризоване зображення
    binary_mask = (mask > 0.5).astype(np.uint8) * 255

    # Шлях до зображення, яке передав Gradio
    try:
        filename = image.info['filename']  
    except:
        filename = "unknown.png"

    # Застосування правил із датасету MnM
    if use_rules:
        threshold, morphology = get_rules_for_image(filename)

        # Застосування морфології згідно з правилами
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        if morphology == 'open-close':
            binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
            binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
        elif morphology == 'close':
            binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
        elif morphology == 'open':
            binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
       

    # Візуалізація
    img_rgb = np.array(image.convert("RGB").resize((256, 256)))
    overlay = img_rgb.copy()
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    palette = [(0,0,255),(0,255,0),(255,0,0),(0,255,255),(255,0,255),(255,255,0)]
    alpha = 0.5

    for i, cnt in enumerate(contours):
        color = palette[i % len(palette)]
        mask = np.zeros_like(binary_mask)
        cv2.drawContours(mask, [cnt], -1, 255, thickness=cv2.FILLED)
        for c in range(3):
            overlay[:,:,c] = np.where(mask==255,
                                      overlay[:,:,c]*(1-alpha) + color[c]*alpha,
                                      overlay[:,:,c])
    out_img = cv2.addWeighted(overlay, alpha, img_rgb, 1 - alpha, 0)
    return Image.fromarray(out_img.astype(np.uint8))

# Gradio інтерфейс
demo = gr.Interface(
    fn=segment_mri,
    inputs=[
        gr.Image(type="pil", label="Зображення МРТ серця"),
        gr.Checkbox(label="Інтегрувати правила з датасету MnM", value=True)
    ],
    outputs=gr.Image(type="pil", label="Результат сегментації"),
    title="Сегментація МРТ серця з інтеграцією знань з MnM",
    submit_btn="Опрацювати",
    clear_btn="Очистити"
)

demo.launch(inline=True)


c:\super_krb\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\super_krb\.venv\Lib\site-packages\gradio\interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
